# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.80/12607/1 Dashboard: http://10.20.0.80:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,955,Tim,-0.39387284626670294,-0.6955519199428926
2000-01-01 00:00:01,997,Bob,-0.4865597632613108,0.9678173550305893
2000-01-01 00:00:02,981,Norbert,0.4153873238300443,0.22055177784128177
2000-01-01 00:00:03,1027,Kevin,0.04387622904764532,0.3653452536034061
2000-01-01 00:00:04,971,Hannah,-0.2309522564228561,0.0946792698170904
2000-01-01 00:00:05,1053,Edith,-0.5455432938402145,-0.8757825923334548
2000-01-01 00:00:06,1005,Zelda,-0.004861157168762231,0.7838135088474654
2000-01-01 00:00:07,1023,Hannah,0.7134124791139733,0.5466272132676708
2000-01-01 00:00:08,977,Bob,-0.7477289369995717,0.8191332519980414


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1029,Xavier,0.23857854569900505,-0.4511593826319378
2000-01-30 00:00:01,999,Jerry,0.7288813631984705,-0.48864515899884586
2000-01-30 00:00:02,905,Charlie,-0.2939064886013152,-0.5570131161073997
2000-01-30 00:00:03,1033,Alice,-0.6591225545840596,-0.15263254575007257
2000-01-30 00:00:04,932,Alice,0.6193785637550029,-0.7358109986037338
2000-01-30 00:00:05,995,Patricia,0.20741789228669094,0.039058311553305414
2000-01-30 00:00:06,966,Bob,0.41003018319549267,0.6799678363322479
2000-01-30 00:00:07,1045,Kevin,-0.4405427568347102,0.19873043656378897
2000-01-30 00:00:08,944,Norbert,0.7841608268979066,-0.7414975077779835


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,955,Tim,-0.393873,-0.695552
1,2000-01-01 00:00:01,997,Bob,-0.486560,0.967817
2,2000-01-01 00:00:02,981,Norbert,0.415387,0.220552
3,2000-01-01 00:00:03,1027,Kevin,0.043876,0.365345
4,2000-01-01 00:00:04,971,Hannah,-0.230952,0.094679


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,955,Tim,-0.393873,-0.695552
1,2000-01-01 00:00:01,997,Bob,-0.486560,0.967817
2,2000-01-01 00:00:02,981,Norbert,0.415387,0.220552
3,2000-01-01 00:00:03,1027,Kevin,0.043876,0.365345
4,2000-01-01 00:00:04,971,Hannah,-0.230952,0.094679


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.67 s, sys: 520 ms, total: 5.19 s
Wall time: 3.39 s


name
Alice       0.002068
Bob        -0.001588
Charlie    -0.004001
Dan        -0.002517
Edith      -0.000939
Frank      -0.004115
George     -0.000490
Hannah      0.000429
Ingrid      0.002605
Jerry      -0.001425
Kevin      -0.001907
Laura       0.005139
Michael    -0.003753
Norbert     0.001480
Oliver      0.001196
Patricia   -0.001855
Quinn       0.000317
Ray         0.001956
Sarah       0.001771
Tim        -0.002986
Ursula     -0.000227
Victor      0.000388
Wendy       0.000913
Xavier      0.001811
Yvonne      0.001174
Zelda       0.000497
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.49 s, sys: 124 ms, total: 1.62 s
Wall time: 1.09 s


name
Alice       0.002068
Bob        -0.001588
Charlie    -0.004001
Dan        -0.002517
Edith      -0.000939
Frank      -0.004115
George     -0.000490
Hannah      0.000429
Ingrid      0.002605
Jerry      -0.001425
Kevin      -0.001907
Laura       0.005139
Michael    -0.003753
Norbert     0.001480
Oliver      0.001196
Patricia   -0.001855
Quinn       0.000317
Ray         0.001956
Sarah       0.001771
Tim        -0.002986
Ursula     -0.000227
Victor      0.000388
Wendy       0.000913
Xavier      0.001811
Yvonne      0.001174
Zelda       0.000497
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.28 s, sys: 89.7 ms, total: 1.37 s
Wall time: 959 ms


name
Alice       0.002068
Bob        -0.001588
Charlie    -0.004001
Dan        -0.002517
Edith      -0.000939
Frank      -0.004115
George     -0.000490
Hannah      0.000429
Ingrid      0.002605
Jerry      -0.001425
Kevin      -0.001907
Laura       0.005139
Michael    -0.003753
Norbert     0.001480
Oliver      0.001196
Patricia   -0.001855
Quinn       0.000317
Ray         0.001956
Sarah       0.001771
Tim        -0.002986
Ursula     -0.000227
Victor      0.000388
Wendy       0.000913
Xavier      0.001811
Yvonne      0.001174
Zelda       0.000497
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html